# EU Data compiler

**Option 1 Python EU lex project (https://pypi.org/project/eurlex/)**

In [2]:
#get the neccassary package
! pip install eurlex

  Obtaining dependency information for eurlex from https://files.pythonhosted.org/packages/ed/3c/c869754cd09e38f698e0813261c7638ae454bef51c1702df004168d79c83/eurlex-0.1.4-py3-none-any.whl.metadata
  Obtaining dependency information for argparse==1.4.0 from https://files.pythonhosted.org/packages/f2/94/3af39d34be01a24a6e65433d19e107099374224905f1e0cc6bbe1fd22a2f/argparse-1.4.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for sparqlwrapper==1.8.5 from https://files.pythonhosted.org/packages/00/9b/443fbe06996c080ee9c1f01b04e2f683b2b07e149905f33a2397ee3b80a2/SPARQLWrapper-1.8.5-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/5.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/5.5 MB ? eta -:--:--
     ---------------------------------------- 0.1/5.5 MB 660.6 kB/s eta 0:00:09
     -- ------------------------------------- 0.4/5.5 MB 3.6 MB/s eta 0:00:02
     ------ --------------------------------- 0.9/5.5 MB 5.6 MB/s eta 0:0

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [1261 lines of output]
  <string>:44: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-311
  creating build\lib.win-amd64-cpython-311\pandas
  copying pandas\conftest.py -> build\lib.win-amd64-cpython-311\pandas
  copying pandas\testing.py -> build\lib.win-amd64-cpython-311\pandas
  copying pandas\_testing.py -> build\lib.win-amd64-cpython-311\pandas
  copying pandas\_typing.py -> build\lib.win-amd64-cpython-311\pandas
  copying pandas\_version.py -> build\lib.win-amd64-cpython-311\pandas
  copying pandas\__init__.py -> build\lib.win-amd64-cpython-311\pandas
  creating build\lib.win-amd64-cpython-311\pandas\api
  copying pandas\api\__init__.py -> build\lib.win-amd64-cpython-311\pandas\api
  creating build\lib.win-amd64-cpython-311\pandas\arrays
  copying pandas\arrays\__i

*in the next step, take the keyword list and search for each keyword in the EU Lex database, extracct the CELEX numbers of each document for ech search as zipped csv file and compile a list of unique CELEX numbers*

In [3]:
#take the CELEX IDs queried through the keyword searches and download all documents
import eurlex
from eurlex import get_html_by_celex_id, parse_html

# Retrieve and parse the document with CELEX ID "32019R0947" into a Pandas DataFrame
celex_id = "32019R0947"
html = get_html_by_celex_id(celex_id)
df = parse_html(html)

# Get the first line of Article 1
df_article_1 = df[df.article == "1"]
df_article_1_line_1 = df_article_1.iloc[0]

# Display the subtitle of Article 1
print(df_article_1_line_1.article_subtitle)


# Display the corresponding text
print(df_article_1_line_1.text)

ModuleNotFoundError: No module named 'eurlex'

**Option 2 Python EU lex SPARQL query (https://pypi.org/project/pyeurlex/)**

In [ ]:
! pip install pyeurlex

In [ ]:
from eurlex import Eurlex
eur = Eurlex()

In [ ]:
#create a query based on the advanced expert search SPARQL query format of the eulex website
q = eur.make_query(resource_type = "caselaw", order = True, limit = 10)
print(q)

In [ ]:
d = eur.query_eurlex(q)  # where q is a query generated in a previous step or a string defined by you
print(d)

*This will return a pandas data frame of the results. Its columns depend on the the fields that you included. 
Once you pick a single url or identifier from the df, you can download a notice or data based on that indentifier. To download the notices as xml, use download_xml() as below.*


*This step will need to be done in a loop to access and download all the files*

In [ ]:
x = eur.download_xml("32014R0001", notice="tree") # without the file parameter to specify the filename, the celex number will be used.
print(x)

#get thedata
d = eur.get_data("http://publications.europa.eu/resource/celex/32016R0679", type="text")
print(d)

**Option3: SPARQL query via EU Lex Expert Search**

**Option 4: LEGALBERT**